# Pancakeswap momentum trading

This is an example strategy backtesting how to momentum trading on PancakeSwap.

## Creating trading universe

First let's import libraries and initialise our dataset client.

In [4]:
try:
    import tradingstrategy
except ImportError:
    !pip install tradingstrategy
    import site
    site.main()

from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


Let's create a pair universe for PancakeSwap.
We will create a dataset of 2d candles that trade on PancakeSwap on Binance Smart Chain.

In [5]:
import pandas as pd
from tradingstrategy.chain import ChainId
from tradingstrategy.pair import PandasPairUniverse
from tradingstrategy.timebucket import TimeBucket

columnar_pair_table = client.fetch_pair_universe()

exchange_universe = client.fetch_exchange_universe()

all_pairs_dataframe = columnar_pair_table.to_pandas()

our_exchange = exchange_universe.get_by_chain_and_slug(ChainId.bsc, "pancakeswap-v2")
our_pairs: pd.DataFrame = all_pairs_dataframe.loc[
    (all_pairs_dataframe['exchange_id'] == our_exchange.exchange_id) &  # Trades on Sushi
    (all_pairs_dataframe['buy_volume_all_time'] > 500_000)  # 500k min buys
]

# Create a Python set of pair ids
wanted_pair_ids = our_pairs["pair_id"]

# Make the trading pair data easily accessible
pair_universe = PandasPairUniverse(our_pairs)

print(f"Our trading universe has {len(pair_universe.get_all_pair_ids())} pairs that meet the prefiltering criteria")

Our trading universe has 5357 pairs that meet the prefiltering criteria


Get daily candles and filter them against our wanted pair set.

In [6]:
import datetime

from tradingstrategy.candle import GroupedCandleUniverse
from tradingstrategy.pair import PandasPairUniverse
from tradingstrategy.frameworks.backtrader import prepare_candles_for_backtrader

TARGET_TIME_BUCKET = TimeBucket.d1

# Get daily candles as Pandas DataFrame
all_candles = client.fetch_all_candles(TARGET_TIME_BUCKET).to_pandas()
our_candles: pd.DataFrame = all_candles.loc[all_candles["pair_id"].isin(wanted_pair_ids)]

our_candles = prepare_candles_for_backtrader(sushi_candles)

# We limit candles to a specific date range to make this notebook deterministic
start = datetime.datetime(2020, 10, 1)
end = datetime.datetime(2022, 1, 1)

our_candles = sushi_candles[(sushi_candles.index >= start) & (sushi_candles.index <= end)]

# Group candles by the trading pair ticker
our_candle_universe = GroupedCandleUniverse(sushi_candles)

print(f"Total number of candles we use in the simulation is {len(our_candles)}")

  0%|          | 0/226746797 [00:00<?, ?it/s]

NameError: name 'sushi_candles' is not defined

## Creating the strategy

[See the Backtrader quickstart tutorial](https://www.backtrader.com/docu/quickstart/quickstart/).

### Cumulative volume indicator

We create a [Backtrader indicator based on PeriodN indicator](https://github.com/mementum/backtrader/blob/0fa63ef4a35dc53cc7320813f8b15480c8f85517/backtrader/indicators/basicops.py) how much cumulative trade volume the trading pair has seen in the past. We use this indicator to filter out trading pairs that seem to be dead, as no trading happening, and thus not subject to our random entry.

More information about [Backtrader custom indicators](https://www.backtrader.com/docu/inddev/).

In [ ]:
import math
from random import Random

import backtrader as bt
from backtrader.indicators import PeriodN


class PastTradeVolumeIndicator(PeriodN):
    """Indicates whether the trading pair has reached certain volume for the last N days.

    Based on indicator base class that takes period (days) as an input.
    """

    lines = ('cum_volume',)

    params = (('period', 7),)

    def next(self):
        # This indicator is feed with volume line.
        # We simply take the sum of the daily volumes based on the period (number of days)
        datasum = math.fsum(self.data.get(size=self.p.period))
        self.lines.cum_volume[0] = datasum


### Coinflip strategy core

By using the indicator from the above here is our strategy.

In [ ]:
from tradingstrategy.frameworks.backtrader import CapitalgramFeed

class EntropyMonkey(bt.Strategy):
    """A strategy that picks a new token to go all-in every day."""

    def __init__(self, pair_universe: PandasPairUniverse, seed: int):
        #: Allows us to print human-readable pair information
        self.pair_universe = pair_universe

        #: Initialize (somewhat) determininistic random number generator
        self.dice = Random(seed)

        #: We operate on daily candles.
        #: At each tick, we process to the next candle
        self.day = 0

        #: Cumulative volume indicator for each of the data feed
        self.indicators = {}
        pair: CapitalgramFeed
        for pair in self.datas:
            self.indicators[pair] = PastTradeVolumeIndicator(pair.lines.volume)

        # How much USD volume token needs to have in order to be eligible for a pick
        self.cumulative_volume_threshold = 200_000

        # How many times we try to pick a token pair to buy
        # before giving up (at early days there might not be enough volume)
        self.pick_attempts = 100

        # If our balance goes below this considering giving up
        self.cash_balance_death_threshold = 100

    def next(self):
        """Tick the strategy.

        Because we are using daily candles, tick will run once per each day.
        """

        # Advance to the next day
        self.day += 1

        # Pick a new token to buy
        for i in range(self.pick_attempts):
            random_pair: CapitalgramFeed = self.dice.choice(self.datas)
            pair_info = random_pair.pair_info
            cum_volume_indicator = self.indicators[random_pair]
            volume = cum_volume_indicator.lines.cum_volume[0]
            if volume > self.cumulative_volume_threshold and random_pair.close[0] > 0:
                break
        else:
            print(f"On day #{self.day} did not find any token to buy")
            return

        # Sell any existing token we have
        for ticker in self.datas:
             if self.getposition(ticker).size > 0:
                # Too verbose
                # print(f"On day #{self.day}, selling existing position of {pair_info.base_token_symbol} - {pair_info.quote_token_symbol}")
                self.close(ticker)

        # Buy in with all money we have.
        # We are not really worried about order size quantisation in crypto.
        cash = self.broker.get_cash()

        if cash < self.cash_balance_death_threshold:
            # We are busted
            return

        # Buy using the daily candle closing price as the rate
        price = random_pair.close[0]
        assert price > 0, "Pair had zero price "
        size = cash / price

        # Sell the existing position
        print(f"On day #{self.day} we are buying {pair_info.base_token_symbol} - {pair_info.quote_token_symbol} that has 7 days vol of {volume:,.0f} USD. Buy in at price of {price:.4f} USD, cash at hand {cash:,.2f} USD")

        self.buy(random_pair, size=size, exectype=bt.Order.Market)


## Feeding the strategy

Feed our trade data from Sushiswap to the Backtrader strategy.

In [ ]:
from tradingstrategy.frameworks.backtrader import add_dataframes_as_feeds

# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)

# Add a strategy
cerebro.addstrategy(EntropyMonkey, pair_universe=pair_universe, seed=0x1000)

# Pass all Sushi pairs to the data fees to the strategy
# noinspection JupyterKernel
feeds = [df for pair_id, df in sushi_tickers.get_all_pairs()]
add_dataframes_as_feeds(
    cerebro,
    pair_universe,
    feeds,
    start,
    end,
    TimeBucket.d1)

## Running the strategy

We are adding some observers that tell us how well the strategy performs.

In [ ]:
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)

# Run over everything
print('Starting portfolio value: %.2f USD' % cerebro.broker.getvalue())
results = cerebro.run()
print('Ending portfolio value: %.2f USD' % cerebro.broker.getvalue())

strategy: EntropyMonkey = results[0]

## Analysing the portfolio results

In [ ]:
print(f"Backtest range {start.date()} - {end.date()}")
print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
print(f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%")
print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")

## Plotting the portfolio value

Below is an example chart how the portfolio value moves over the time.

In [ ]:
from matplotlib.figure import Figure

figs = cerebro.plot(iplot=False)

fig: Figure = figs[0][0]

fig

That's all this time. Onwards.


